## Лабораторная работа №2

### Задание 1 
Провести классификацию найденного датасета, методом машины опорных векторов. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Выполнение работы:

#### Шаг 1. Загрузка датасета

In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import pandas as pd

file_path = "../../dataset/result_dataset.csv"
data = pd.read_csv(file_path)

data


,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,1.385424e+09,26,0,2013,19,0,0,0,0,0,0,0,8,45,120,1,360,950
1,1.448496e+09,26,0,2015,19,0,0,0,0,0,0,0,8,45,120,1,360,950
2,1.395533e+09,23,1,2014,49,1,0,1,1,0,0,0,23,45,120,1,1035,2401
3,1.458691e+09,23,1,2016,49,1,0,1,1,0,0,0,20,45,120,1,900,2088
4,1.400112e+09,15,2,2014,47,1,1,1,1,0,0,0,4,45,120,0,180,418
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113031,1.460419e+09,12,11,2016,41,1,0,5,9,1,16,129,3,24,64,0,72,184
113032,1.396397e+09,2,11,2014,18,0,0,1,8,1,16,128,22,24,64,1,528,1183
113033,1.459555e+09,2,11,2016,18,0,0,1,8,1,16,128,22,24,64,1,528,1183
113034,1.393891e+09,4,1,2014,37,1,1,4,17,1,16,127,24,24,64,1,576,1260


#### Шаг 2. Разделение данных и нормализация

In [2]:
# Разделение на признаки (X) и целевую переменную (y)
X = data.drop('Profit', axis=1)
y = data['Profit']

# Разделение данных на тренировочный и тестовый наборы
# random_state=42 - гарантирует, что данные каждый раз будут одинакого разбиваться
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Масштабирование признаков (нормализация)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Размер обучающего набора: {X_train_scaled.shape}")
print(f"Размер тестового набора: {X_test_scaled.shape}")

Размер обучающего набора: (90428, 17)
Размер тестового набора: (22608, 17)


#### Шаг 3. Гиперпараметры:

C (регуляризация): Определяет силу регуляризации в модели SVM (Support Vector Machine / Машина опорных векторов); более низкое значение C увеличивает штраф за неправильную классификацию, что может привести к созданию простой модели с большей обобщающей способностью.
gamma (ядерный коэффициент): Определяет влияние одного тренировочного примера на другие; меньшие значения gamma означают более широкие гауссовы функции и приводят к более простым/прямым границам решения.
kernel (ядро): Определяет тип использованного ядра (линейное или RBF); линейное ядро работает линейно, тогда как RBF (радиально-базисная функция) способно моделировать сложные границы решений.

In [3]:
param_grid = {'C': [0.01, 0.1, 1], 'kernel': ['linear', 'rbf', 'sigmoid'], 'gamma': [0.1, 1, 10]}

#### Шаг 4. Подбор гиперпараметров с помощью перекрестной проверки

In [4]:
grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
grid.fit(X_train_scaled, y_train)

best_params = grid.best_params_
best_score = grid.best_score_

Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.962 total time=  21.1s
[CV 2/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.963 total time=  21.4s
[CV 3/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.960 total time=  24.0s
[CV 4/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.958 total time=  24.1s
[CV 5/5] END ..C=0.01, gamma=0.1, kernel=linear;, score=0.958 total time=  24.1s
[CV 1/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.877 total time= 1.8min
[CV 2/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.878 total time= 1.8min
[CV 3/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.875 total time= 1.8min
[CV 4/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.879 total time= 1.9min
[CV 5/5] END .....C=0.01, gamma=0.1, kernel=rbf;, score=0.874 total time= 1.9min
[CV 1/5] END .C=0.01, gamma=0.1, kernel=sigmoid;, score=0.844 total time= 2.5min
[CV 2/5] END .C=0.01, gamma=0.1, kernel=sigmoid

#### Шаг 5. Вывод лучших гиперпараметров и оценка производительности модели на тестовом наборе

In [5]:
print("Лучшие гиперпараметры:", grid.best_params_)
best_svc = grid.best_estimator_
y_pred = best_svc.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print(f"Точность модели: {accuracy}")

# Получение отчета о классификации с предупреждениями
report = classification_report(y_test, y_pred, zero_division=1)
print(f'Отчет о классификации:\n{report}')

Лучшие гиперпараметры: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Точность модели: 0.9974345364472753
Отчет о классификации:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     15691
           1       1.00      0.99      1.00      6917

    accuracy                           1.00     22608
   macro avg       1.00      1.00      1.00     22608
weighted avg       1.00      1.00      1.00     22608

